## Training

In [13]:
import tarfile
from pathlib import Path
#from data import create_dir, download_data
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import  transforms
import glob
import numpy as np
import os
import random
import shutil
import tarfile
import torch
import torchvision.transforms.functional as TF


# setup directory structure for download

#process_dir = Path("/Volumes/KINGSTON/Remote Sensing Presentation/Test")

#list(process_dir.glob(".gz"))

In [ ]:
args = {
    "batch_size": 10, # make this bigger if you are not running on binder
    "epochs": 10,
    "lr": 0.0001,
    "device": "cpu" # set to "cuda" if GPU is available
}


In [18]:
#for x in list(process_dir.glob("*.gz")):
    #tar = tarfile.open(x)
    #tar.extractall(x.parent)
    #tar.close()    

In [19]:
class GlacierDataset(Dataset):
    """Custom Dataset for Glacier Data

    Indexing the i^th element returns the underlying image and the associated
    binary y
    """
    def __init__(self, x_paths, y_paths, imsize=512):
        self.x_paths = x_paths
        self.y_paths = y_paths
        self.imsize = imsize

    def __getitem__(self, index):
        x_path = self.x_paths[index]
        y_path = self.y_paths[index]
        z = [np.load(p) for p in [x_path, y_path]]
        z = [TF.to_tensor(z_).float() for z_ in z]

        # Random crop
        i, j, h, w = transforms.RandomCrop.get_params(z[0], output_size=(self.imsize, self.imsize))
        z = [TF.crop(z_, i, j, h, w) for z_ in z]

        # Random flipping
        if random.random() > 0.5:
            z = [TF.hflip(z_) for z_ in z]

        if random.random() > 0.5:
            z = [TF.vflip(z_) for z_ in z]

        return z


    def __len__(self):
        return len(self.x_paths)



NameError: name 'Dataset' is not defined

In [ ]:
from torch.utils.data import DataLoader
process_dir =Path("./Processed/Train")
paths = {
    "x": list((process_dir).glob("x*")),
    "y": list((process_dir).glob("y*"))
}
ds = GlacierDataset(paths["x"], paths["y"])
loader = DataLoader(ds, batch_size=args["batch_size"], shuffle=True)

In [ ]:
import torch.optim
from unet import Unet
from train import train_epoch

model = Unet(13, 3, 4, dropout=0.2).to(args["device"])
optimizer = torch.optim.Adam(model.parameters(), lr=args["lr"])

for epoch in range(args["epochs"]):
    train_epoch(model, loader, optimizer, args["device"], epoch)
    
torch.save(model.state_dict(), data_dir / "model.pt")